In [1]:
library("IBMPopSim")
library("ggplot2")
library("Rcpp")

Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


Attaching package: ‘IBMPopSim’


The following object is masked from ‘package:stats’:

    stepfun




In [2]:
# Generate initial population
N <- 1000  # Number of individuals in the initial population

In [3]:
# xb - xd is distributed uniformly on -10 ,10
xd <- runif(N,0,10)
pop_init <- data.frame(
  "birth" = rep(0,N), 
  "death" = as.double(NA),
  "lancing" = rep(c(FALSE,TRUE),N/2),
    "xb"= 10-xd, 
    "xd" =  xd, 
    "lignee"=1:N
)
get_characteristics(pop_init)


lancing       xb       xd   lignee 
  "bool" "double" "double"    "int"

In [4]:
head(pop_init)
#plot.ecdf(pop_init$xb-pop_init$xd)

,birth,death,lancing,xb,xd,lignee
,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<int>
1,0,NA,FALSE,9.227705,0.7722946,1
2,0,NA,TRUE,3.996699,6.0033010,2
3,0,NA,FALSE,1.773521,8.2264786,3
4,0,NA,TRUE,8.641576,1.3584240,4
5,0,NA,FALSE,4.079241,5.9207590,5
6,0,NA,TRUE,8.275549,1.7244514,6


In [16]:
# Events and model creation
 # There are 2 possible events :

# - Birth (with or without mutation)
# - Death

# Each event is characterized by its intensity and  kernel code, described below.

## Birth event with individual intensity

### Parameters



In [6]:
params_birth <- list("ib"= 1, "p"=0.1 , "var_mut"=0.05)

In [7]:

birth_event <- mk_event_individual( type = "birth",
  intensity_code = ' if (age(I,t) < I.xb)
                        result = ib; 
                    else 
                        result =0;',  # each individual  I can give birth at rate ib if its age is less than xb
  kernel_code = 'if (CUnif() < p)
                     newI.xb = max(0., CNorm(I.xb, var_mut)); 
                 else
                     newI.xb = I.xb;
                if (I.lancing & (age(I,t)> I.xd) & (age(I,t)<I.xb))
                     newI.xd =0;
                 else{
                    if (CUnif()<p)
                        newI.xd =max(0., CNorm(I.xd, var_mut));
                     else 
                        newI.xd =I.xd;}
                 newI.lancing =I.lancing;
                 newI.lignee =I.lignee;') 
# An individual I can give birth to an individual newI. The kernel code defines characteristics of individual newI
# Attention la manière dont est calculée le trait après mutation est un peu différente du code du Tristan

In [8]:
## Death event 
### parameters
params_death <- list("intens"=1, "compet"= 0.0009)

In [9]:
## Deaths due to interactions
death_event1 <- mk_event_interaction(name='death1',
  type = "death",
  interaction_code = "result = compet;" 
)

In [10]:
## Deaths due to aging 
death_event2 <- mk_event_individual(name='death2', type="death",
                  intensity_code = ' if (age(I,t)>I.xd) result= intens; 
                                     else result =0;')

In [11]:
# Model creation 
model <- mk_model(
  characteristics = get_characteristics(pop_init),
  events = list(birth_event, death_event1, death_event2),
  parameters = c(params_birth, params_death)

)
summary(model)

x86_64-conda_cos6-linux-gnu-c++ -std=gnu++11 -I"/opt/conda/lib/R/include" -DNDEBUG -I../inst/include -fopenmp  -I"/opt/conda/lib/R/library/Rcpp/include" -I"/home/criparis/R/x86_64-conda_cos6-linux-gnu-library/3.6/IBMPopSim/include" -I"/opt/conda/lib/R/library/RcppArmadillo/include" -I"/tmp/RtmpOJ57U7/sourceCpp-x86_64-conda_cos6-linux-gnu-1.0.1" -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib  -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/tmp/build/80754af9/r-base_1589917437985/work=/usr/local/src/conda/r-base-3.6.1 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c file326d14cdd4ee0.cpp -o file326d14cdd4ee0.o


ERROR: Error in sourceCpp(code = model$cpp_code): Error 1 occurred building shared library.




Please install GNU development tools including a C++ compiler.




In [12]:
## Bounds for birth and death rates 

birth_intensity_max <- params_birth$ib
interaction_fun_max <- params_death$compet
death2_max <- params_death$intens

In [13]:
T = 150 # Simulation end time 


sim_out <- popsim(model = model,
  population = pop_init,
  events_bounds = c('birth'=birth_intensity_max, 'death1'=interaction_fun_max,'death2'= death2_max),
  parameters = c(params_birth, params_death),
  time = T)

Simulation on  [0, 150] 

In [14]:
# Simulation with different parameters

#The model can be simulated with different parameters without being recompiled.


In [15]:
#sim_out$logs["duration_main_algorithm"]
#sim_out$logs

In [16]:
# Outputs

In [17]:
str(sim_out$population)

'data.frame':	164054 obs. of  6 variables:
 $ birth  : num  143 144 144 145 145 ...
 $ death  : num  NA NA NA NA NA NA NA NA NA NA ...
 $ lancing: logi  TRUE FALSE TRUE TRUE FALSE FALSE ...
 $ xb     : num  4.66 6.06 5.64 4.65 5.89 ...
 $ xd     : num  5.61 4.52 4.63 5.43 4.1 ...
 $ lignee : int  638 969 46 638 425 425 425 425 425 642 ...


In [20]:
pop_out <- sim_out$population
head(pop_out)

,birth,death,lancing,xb,xd,lignee
,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<int>
1,143.3926,NA,TRUE,4.658894,5.605821,638
2,143.7459,NA,FALSE,6.060446,4.522422,969
3,143.8933,NA,TRUE,5.644276,4.630242,46
4,144.9153,NA,TRUE,4.654941,5.425354,638
5,144.9893,NA,FALSE,5.886931,4.100464,425
6,145.1161,NA,FALSE,5.977769,3.791014,425


In [ ]:
pop_size <- nrow(population_alive(pop_out,t = 150))

xbxd_evol_for_publi <- ggplot(pop_out, aes(color=as.logical(Lansing), alpha = 0.1)) + 
  geom_segment(aes(x=birth, xend=death, y=xb-xd, yend=xb-xd, alpha =0.1) , na.rm=TRUE)+
  xlab("Time") +
  ylab("xb-xd") + 
  geom_hline(yintercept=log(3)/2, color = "red", linetype="dashed") +
  geom_hline(yintercept = 0, color = "blue", linetype="dashed") +
  theme(legend.position="none")

xbxd_evol_for_publi